<img src="UNAV.png" width=100 height=100 align="right">

# Python Ecosystem -  Trabajando con ficheros de datos - Numpy & Pandas

<img src="pwc_logo.png" width=120 height=120 align="right">

<img src="profile_manoelgadi.png" width=100 height=100 align="right">

Author: Prof. Manoel Gadi

Contact: manoelgadi@gmail.com

Teaching Web: http://mfalonso.pythonanywhere.com

Last revision: 18/June/2019

Date: 07/February/2018
    
Author: Prof. Manoel Gadi - http://mfalonso.pythonanywhere.com/ - https://www.linkedin.com/in/manoel-gadi-97821213/


## Parte 2: Trabajando con Data Frames para calcular la Beta de mercado


<img src="BetaCalculation.png" align="center">




* Referencia 1: http://gouthamanbalaraman.com/blog/calculating-stock-beta.html
* Referencia 2: http://www.investopedia.com/ask/answers/070615/what-formula-calculating-beta.asp

In [2]:
!pip install pandas_datareader

In [3]:
!pip install fix_yahoo_finance

  Created wheel for fix-yahoo-finance: filename=fix_yahoo_finance-0.1.35-py3-none-any.whl size=12443 sha256=006b27f6700975844c46fe863488aa6d494945766438167d778c912efa3f3971
  Stored in directory: c:\users\aulasfarmacia\appdata\local\pip\cache\wheels\ec\8b\42\b4b6c3019ecbb6e352f974109b8c971bdf7f9dba0278997aca
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=f8cccb64b353f60fd5fa58c993a28b8554103b432b9b1fb3f0fd4e1295a64bba
  Stored in directory: c:\users\aulasfarmacia\appdata\local\pip\cache\wheels\f2\b5\2c\59ba95dcf854e542944c75fe3da584e4e3833b319735a0546c
Successfully built fix-yahoo-finance multitasking


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\aulasfarmacia\Anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\aulasfarmacia\\AppData\\Local\\Temp\\pip-install-b6srz_eu\\fix-yahoo-finance_df06df4822f34c2c935a51eb59479d68\\setup.py'"'"'; __file__='"'"'C:\\Users\\aulasfarmacia\\AppData\\Local\\Temp\\pip-install-b6srz_eu\\fix-yahoo-finance_df06df4822f34c2c935a51eb59479d68\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\aulasfarmacia\AppData\Local\Temp\pip-pip-egg-info-otbgta1r'
         cwd: C:\Users\aulasfarmacia\AppData\Local\Temp\pip-install-b6srz_eu\fix-yahoo-finance_df06df4822f34c2c935a51eb59479d68\
    Complete output (7 lines):
    Traceback (most recent c

In [4]:
import pandas_datareader.data as web
import fix_yahoo_finance as yf
import datetime
import numpy as np
import pandas as pd

### Descargando datos historicos

Capturando datos de serie historica de 5 años para la acción de Telefónica (TEF.MC) y para el índice IBEX 35

In [7]:
fecha_final =   datetime.datetime(2019,12, 31, 0, 0, 0, 0)
fecha_inicial = fecha_final - datetime.timedelta(days=5*365)

In [8]:
df = web.get_data_yahoo("SPY", start=fecha_inicial, end=fecha_final)

In [ ]:
fecha_final, fecha_inicial

In [ ]:
ticker_symbol = 'TEF.MC'
ref_index = '^IBEX'

ticker_symbol, ref_index

Leyendo los datos de la API de https://finance.yahoo.com/?guccounter=1

In [ ]:
df_stock = web.DataReader(ticker_symbol,'yahoo',fecha_inicial,fecha_final)

In [ ]:
df_stock.head()

Haciendo lo mismo para el indice de referencia

In [ ]:
df_index = web.DataReader(ref_index,'yahoo',fecha_inicial,fecha_final)

In [ ]:
df_index.head()

### Para calcular la Beta necesitamos crear una serie temporal de puntos de datos mensuales

In [ ]:
df_stock = df_stock.resample('M').last()

In [ ]:
df_stock.head()

Haciendo lo mismo para el indice de referencia

In [ ]:
df_index = df_index.resample('M').last()

In [ ]:
df_index.head()

### Para calcular la Beta utilizamos el percentual de variación mes a mes, no el valor de la acción directamente

In [ ]:
df_stock['returns'] = df_stock['Adj Close']/ df_stock['Adj Close'].shift(1) -1

In [ ]:
df_stock['returns'].head()

In [ ]:
df_stock = df_stock.dropna()

In [ ]:
df_stock['returns'].head()

In [ ]:
df_stock.head()

Haciendo lo mismo para el indice de referencia

In [ ]:
df_index['returns'] = df_index['Adj Close']/ df_index['Adj Close'].shift(1) -1
df_index = df_index.dropna()


In [ ]:
df_index.head()

### Haciendo merge de los 2 dataframes por fecha (el index del dataframe es la fecha) y cogiendo solo las columnas stock_returns y index_returns

In [ ]:
df = pd.DataFrame({'stock_returns' : df_stock['returns'],
                        'index_returns' : df_index['returns']},
                        index=df_stock.index)

In [ ]:
df.head()

Por segurida borramos las filas que no tienen los 2 valores por si hubiera.

In [ ]:
df = df.dropna()

In [ ]:
df.head()

#### Implementando la función para la covarianza

Referencia: http://ci.columbia.edu/ci/premba_test/c0331/s7/s7_5.html

In [ ]:
def covariance(a, b):
    if len(a) != len(b):
        return
    a_mean = np.mean(a)
    b_mean = np.mean(b)
    sum = 0
    for i in range(0, len(a)):
        sum += ((a[i] - a_mean) * (b[i] - b_mean))
    return sum/(len(a)-1)

Miremos otra vez a la formula de la Beta:
    
<img src="BetaCalculation.png" width=400 height=400 align="center">

In [ ]:
numerator = covariance(df['stock_returns'],df['index_returns'])
print("COVARIANCE(stock, benchmark) = COVARIANCE("
      +ticker_symbol+", "+ref_index +") = " +str(numerator))

In [ ]:
denominator = covariance(df['index_returns'],df['index_returns'])
print("VARIANCE(benchmark) = COVARIANCE(benchmark, benchmark) = COVARIANCE("
      +ref_index+", "+ref_index +") = " +str(denominator))


In [ ]:
# BETA = Covariance (stock,index) / Variance (Index) = Covariance (stock,index) / Covariance (stock,stock)
print("BETA = COVARIANCE(stock, benchmark) / VARIANCE(benchmark) = " 
      + str(numerator) + " / " + str(denominator) + " = " 
      +str(covariance(df['stock_returns'],df['index_returns'])
           /covariance(df['index_returns'],df['index_returns'])))

<img src="curso.jpeg" width=300 height=300 align="center">

<img src="UNAV.png" width=100 height=100 align="right">

<img src="pwc_logo.png" width=120 height=120 align="right">